# Deploy a un endpoint online

## Conectar a workspace

In [ ]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

## Definir y crear un endpoint

Definir el endpoint. 
> Tener en cuenta que el nombre debe ser único: Usaremos la función `dateTime` para ello.

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

# online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

online_endpoint_name = 'my-deployment-endpoint-ric'

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for MLflow diabetes model",
    auth_mode="key",
)

Crear el endpoint:

In [ ]:
ml_client.begin_create_or_update(endpoint).result()

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> ¡IMPORTANTE!¡Esperar hasta que el endpoint se cree con éxito antes de continuar!
Una notificación verde debería aparecer en el estudio. </p>



## Configurar el deployment

### Modelo local

In [ ]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

# Alternativa: Obtener el modelo de la ruta local
model = Model(
    path="./model",
    type=AssetTypes.MLFLOW_MODEL,
    description="my sample mlflow model",
    name="mlflow-diabetes-model",
)

In [ ]:
# definir el deployment blue
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="STANDARD_DS1_V2",
    instance_count=1,
    # app_insights_enabled=True  # Habilitar Application Insights
)

In [ ]:
# Crear el deployment
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

In [ ]:
# registrar el modelo en el workspace
# ml_client.models.create_or_update(model)

### Modelo registrado

In [ ]:
reg_model=ml_client.models.get(name="mlflow-diabetes-model", version="1")
reg_model.id

In [ ]:
from azure.ai.ml.entities import Environment

# Definir un entorno personalizado
env = Environment(
    name="my-custom-env",
    description="Entorno personalizado",
    image="mcr.microsoft.com/azureml/mlflow-ubuntu18.04-py38-cpu-inference:latest",
    conda_file='./model/conda.yaml',
)

# Registrar el entorno
registered_env = ml_client.environments.create_or_update(env)
print(f"Entorno registrado: {registered_env.name}, Versión: {registered_env.version}")

In [ ]:
curated_envs = ml_client.environments.list()
for env in curated_envs:
    print(f"Nombre: {env.name}, Versión: {env.version}")

In [ ]:
# env = ml_client.environments.get(name="my-custom-env", version="1")
# env

In [ ]:
# definir un nuevo deployment con el entorno personalizado
green_deployment = ManagedOnlineDeployment(
    name="green",
    endpoint_name=online_endpoint_name,
    model=reg_model.id,
    instance_type="STANDARD_DS1_V2",
    instance_count=1,
    # environment=env.id,
)

In [ ]:
# Crear el deployment
ml_client.online_deployments.begin_create_or_update(green_deployment).result()

Asignar el % de tráfico.

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 80}
endpoint.traffic = {"green": 20}
ml_client.begin_create_or_update(endpoint).result()

## Probar el deploymnent

In [ ]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json",
)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")

## Listar los endpoints

In [ ]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

## Obtener detalles de un endpoint

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

## Borrar el endpoint y deployment

In [ ]:
# Eliminar un deployment
ml_client.online_deployments.begin_delete(
    name='green',
    endpoint_name=online_endpoint_name
).result()

In [ ]:
# Eliminar el endpoint
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)